### Text Selection Data Merge

This notebook was used to combine data from the previous notebook.

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 45000  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

sixgram_select_start = 0
sixgram_select_end = 10000

sevengram_select_start = 0
sevengram_select_end = 10000

eightgram_select_start = 0
eightgram_select_end = 10000

ninegram_select_start = 0
ninegram_select_end = 10000

tengram_select_start = 0
tengram_select_end = 10000


# prefix suffix file
prefix_suffix = False  # True, False: True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Result/2-Text Selection Data Merge"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [7]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed  
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [8]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Text Data Read

In [9]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Word_Selected_45000.xlsx")
df_word_select

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
44995,pythons,630
44996,bubbe,630
44997,clucking,630
44998,komal,630


In [10]:
#df_twogram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Twogram_Selected_10000.xlsx")
df_twogram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Twogram_Selected.csv")
#df_twogram_select = df_twogram_select.head(1000)
df_twogram_select

,twogram,frequency
0,i am,9310563
1,do not,8095395
2,it s,7436197
3,that s,4564126
4,you re,4355237
...,...,...
179482,creepy is,203
179483,load for,203
179484,and whipped,203
179485,is oskar,203


In [11]:
#df_threegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Threegram_Selected_10000.xlsx")
df_threegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Threegram_Selected.csv")
df_threegram_select

,threegram,frequency
0,i do not,3246539
1,do not know,1606191
2,what are you,1324981
3,i am not,1120471
4,i am sorry,1103453
...,...,...
871213,about you vi,48
871214,bien phu we,48
871215,every night that,48
871216,moved in daylight,48


In [12]:
#df_fourgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fourgram_Selected_10000.xlsx")
df_fourgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fourgram_Selected.csv")
df_fourgram_select

,fourgram,frequency
0,i do not know,1317934
1,what are you doing,665946
2,i do not want,310217
3,what s going on,298508
4,what do you mean,278296
...,...,...
1844830,surgically grafted to his,25
1844831,whatever snow does in,25
1844832,was plain to see,25
1844833,on that survival course,25


In [13]:
#df_fivegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fivegram_Selected_10000.xlsx")
df_fivegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fivegram_Selected.csv")
df_fivegram_select

,fivegram,frequency
0,i do not want to,182100
1,what are you doing here,167734
2,i do not know what,166508
3,what are you talking about,166083
4,i do not think so,97382
...,...,...
2510517,re about to be lunch,17
2510518,kittredge is not for you,17
2510519,how about other than ralph,17
2510520,he s more than qualified,17


In [14]:
#df_sixgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sixgram_Selected_10000.xlsx")
df_sixgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sixgram_Selected.csv")
df_sixgram_select

,sixgram,frequency
0,i do not know what to,43489
1,what the hell are you doing,43421
2,i do not know what you,41846
3,do not know what you re,39759
4,let s get out of here,36588
...,...,...
2671673,am here well i can not,12
2671674,it s a model like an,12
2671675,we d be interested to see,12
2671676,you got yourself into those cuffs,12


In [15]:
#df_sevengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sevengram_Selected_10000.xlsx")
df_sevengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sevengram_Selected.csv")
df_sevengram_select

,sevengram,frequency
0,do not know what you re talking,28672
1,i do not know what you re,28558
2,not know what you re talking about,28517
3,what do you want me to do,24163
4,i do not know what to do,17944
...,...,...
1823499,a doctor will see you in mcmurdo,11
1823500,said that we could never never ever,11
1823501,d have to do is read it,11
1823502,you know what i hope you re,11


In [16]:
#df_eightgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Eightgram_Selected_10000.xlsx")
df_eightgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Eightgram_Selected.csv")
df_eightgram_select

,eightgram,frequency
0,do not know what you re talking about,28456
1,i do not know what you re talking,23718
2,i do not want to talk about it,6782
3,have no idea what you re talking about,5211
4,can i talk to you for a second,4847
...,...,...
1139335,the grass it s not even green it,10
1139336,confusion memory loss complete or partial loss of,10
1139337,forget about all that shit this was your,10
1139338,kubrick s next film would prove even more,10


In [17]:
#df_ninegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Ninegram_Selected_10000.xlsx")
df_ninegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Ninegram_Selected.csv")
df_ninegram_select

,ninegram,frequency
0,i do not know what you re talking about,23532
1,i have no idea what you re talking about,4429
2,you do not know what you re talking about,3511
3,what the hell do you think you re doing,3150
4,i do not think that s a good idea,2889
...,...,...
486141,she may be facilitating a terrorist attack on ...,11
486142,need to put a baby in that woman yesterday,11
486143,you ever been sodomized or anything like that ...,11
486144,no they re just doodles you know immature crap,11


In [18]:
#df_tengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Tengram_Selected_10000.xlsx")
df_tengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Tengram_Selected.csv")
df_tengram_select

,tengram,frequency
0,our father who art in heaven hallowed be thy name,3630
1,ten nine eight seven six five four three two one,1772
2,one two three four five six seven eight nine ten,1632
3,hail mary full of grace the lord is with thee,1438
4,captioned by media access group at wgbh access...,1428
...,...,...
144967,you have no idea what your son is going through,20
144968,you worked side by side with adrian chase for ...,20
144969,we re all just trying to do our jobs here,20
144970,the army took the beach from which he had fled,20


#### Text Data Word-Word Group In Word Group

In [19]:
word_list = df_word_select["word"].to_list()
twogram_list = df_twogram_select["twogram"].to_list()
threegram_list = df_threegram_select["threegram"].to_list()
fourgram_list = df_fourgram_select["fourgram"].to_list()
fivegram_list = df_fivegram_select["fivegram"].to_list()
sixgram_list = df_sixgram_select["sixgram"].to_list()
sevengram_list = df_sevengram_select["sevengram"].to_list()
eightgram_list = df_eightgram_select["eightgram"].to_list()
ninegram_list = df_ninegram_select["ninegram"].to_list()
tengram_list = df_tengram_select["tengram"].to_list()

##### Word

In [20]:
# 2 Way Fast
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "twogram"
df_target = df_twogram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [21]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_two = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_word_in_two

,word,twogram
0,lizzie,lizzie borden
1,taller,taller than
2,taller,re taller
3,taller,be taller
4,taller,am taller
...,...,...
314045,gelula,by gelula
314046,gunga,gunga din
314047,meinheimer,dr meinheimer
314048,balon,balon greyjoy


In [22]:
df_word_in_two["word"].nunique()

18850

In [23]:
df_word_in_two_list = df_word_in_two.groupby(["word"])["twogram"].apply(list).reset_index(name="twogram")
df_word_in_two_list

,word,twogram
0,a,"[s a, have a, a little, is a, got a, was a, fo..."
1,aa,"[aa meeting, in aa, an aa]"
2,aah,"[aah aah, ooh aah]"
3,aang,"[aang can, believe aang]"
4,aare,[aare you]
...,...,...
18845,zorn,[danny zorn]
18846,zpm,[the zpm]
18847,zulu,"[yankee zulu, zulu xray]"
18848,zurich,"[in zurich, to zurich]"


In [46]:
#df_word_in_two_list.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)

In [24]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "threegram"
df_target = df_threegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [25]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_three = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_word_in_three

,word,threegram
0,revealing,revealing my true
1,insects,re like insects
2,influence,a bad influence
3,influence,under the influence
4,influence,the influence of
...,...,...
1066358,nosh,a little nosh
1066359,blitzen,cupid donner blitzen
1066360,rambler,in the rambler
1066361,toymaker,the toymaker out


In [26]:
df_word_in_three_list = df_word_in_three.groupby(["word"])["threegram"].apply(list).reset_index(name="threegram")
df_word_in_three_list

,word,threegram
0,a,"[it s a, i am a, you re a, that s a, he s a, t..."
1,aa,"[an aa meeting, aa human human, it s aa, s in aa]"
2,aah,"[aah aah aah, ooh aah ooh, aah ooh ooh, aah ha..."
3,aang,"[aang can save, believe aang can, i believe aang]"
4,aare,"[aare you okay, aare you sure, aare you all]"
...,...,...
25315,zulu,"[yankee zulu xray, zulu six eight, alpha zulu ..."
25316,zurich,"[this is zurich, s in zurich]"
25317,zuzu,[zuzu s petals]
25318,zwei,"[eins zwei drei, ein zwei drei]"


In [27]:
df_word_in_three["word"].nunique()

25320

In [28]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fourgram"
df_target = df_fourgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [29]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_word_in_four

,word,fourgram
0,taller,you re taller than
1,taller,you d be taller
2,taller,i am taller than
3,taller,re taller than i
4,taller,he d be taller
...,...,...
1823198,clucking,even without clucking like
1823199,clucking,without clucking like a
1823200,innovator,i am an innovator
1823201,innovator,the innovator of soul


In [30]:
df_word_in_four_list = df_word_in_four.groupby(["word"])["fourgram"].apply(list).reset_index(name="fourgram")
df_word_in_four_list

,word,fourgram
0,a,"[son of a bitch, i am not a, it s not a, do no..."
1,aa,"[aa human human fucker, i am in aa, he s in aa..."
2,aah,"[aah aah aah aah, ooh aah ooh ooh, be like ooh..."
3,aang,"[aang can save the, i believe aang can, believ..."
4,aare,[aare you all right]
...,...,...
33217,zuko,"[danny zuko turned jock, zuko he s out, on you..."
33218,zulu,"[alpha zulu six eight, zulu six eight niner, s..."
33219,zurg,"[zurg has stolen my, defeated zurg all along, ..."
33220,zurich,"[this is zurich information, he s in zurich, f..."


In [31]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fivegram"
df_target = df_fivegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [32]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_five = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_word_in_five

,word,fivegram
0,aa,the morning sun rises aa
1,aa,aa would be an open
2,aa,after aa would be an
3,aa,uh how about aa snack
4,aa,a couple aa batteries bro
...,...,...
2529439,orientals,orientals they will not give
2529440,orientals,those orientals they will not
2529441,orientals,nobody cares about dead orientals
2529442,coladas,if you like pina coladas


In [33]:
df_word_in_five_list = df_word_in_five.groupby(["word"])["fivegram"].apply(list).reset_index(name="fivegram")
df_word_in_five_list

,word,fivegram
0,a,"[i do not have a, you son of a bitch, i do not..."
1,aa,"[the morning sun rises aa, aa would be an open..."
2,aaa,"[want me to call aaa, those aaa sites were thi..."
3,aaaaah,[will be no more aaaaah]
4,aaah,[me hear you say aaah]
...,...,...
38010,zulu,"[alpha zulu six eight niner, mayday mayday yan..."
38011,zurg,"[could have defeated zurg all, you could have ..."
38012,zurich,"[drive from here to zurich, he s in zurich now..."
38013,zuzu,"[s the matter with zuzu, is that zuzu s teache..."


In [34]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sixgram"
df_target = df_sixgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [35]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_six = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_word_in_six

,word,sixgram
0,revealing,for not revealing my true feelings
1,revealing,on abu ahmed is very revealing
2,revealing,it s been a very revealing
3,revealing,somebody happening by revealing their frailty
4,revealing,i am revealing my true vulnerability
...,...,...
3007583,clucking,two hens would quit your clucking
3007584,clucking,chickens are not clucking around here
3007585,clucking,the chickens are not clucking around
3007586,innovator,not want to be an innovator


In [36]:
df_word_in_six_list = df_word_in_six.groupby(["word"])["sixgram"].apply(list).reset_index(name="sixgram")
df_word_in_six_list

,word,sixgram
0,a,"[it s been a long time, i talk to you for a, i..."
1,aa,"[oh the morning sun rises aa, after aa would b..."
2,aaa,"[you want me to call aaa, s just say i am aaa,..."
3,aaaaah,[there will be no more aaaaah]
4,aaah,[let me hear you say aaah]
...,...,...
40247,zulu,"[mayday mayday mayday yankee zulu xray, s the ..."
40248,zurg,"[could have defeated zurg all along, you could..."
40249,zurich,"[just drive from here to zurich, this is zuric..."
40250,zuzu,"[wwhat s the matter with zuzu, and see what li..."


In [37]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sevengram"
df_target = df_sevengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [38]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_seven = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_word_in_seven

,word,sevengram
0,insects,and he s accompanied by savage insects
1,insects,body was covered in these little insects
2,insects,the insects the mammals the primates monkeys
3,insects,insects do that like spiders and stuff
4,insects,certain insects do that like spiders and
...,...,...
2598575,orientals,those orientals they will not give us
2598576,orientals,orientals they will not give us anything
2598577,orientals,respect bob nobody cares about dead orientals
2598578,orientals,this country s overrun with foreigners orientals


In [39]:
df_word_in_seven_list = df_word_in_seven.groupby(["word"])["sevengram"].apply(list).reset_index(name="sevengram")
df_word_in_seven_list

,word,sevengram
0,a,"[can i talk to you for a, i talk to you for a ..."
1,aa,"[after aa would be an open window, are you han..."
2,aaa,"[let s just say i am aaa, s just say i am aaa ..."
3,aah,"[my life be like ooh aah ooh, life be like ooh..."
4,aand,"[well she was very tired aand uh, aand fix the..."
...,...,...
39590,zuko,"[laid her eyes on you huh zuko, you guys get a..."
39591,zulu,"[that s the way you say zulu, alpha zulu six e..."
39592,zurg,"[buzz you could have defeated zurg all, you co..."
39593,zurich,"[i just drive from here to zurich, this is zur..."


In [40]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "eightgram"
df_target = df_eightgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [41]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_eight = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_word_in_eight

,word,eightgram
0,insects,whole body was covered in these little insects
1,insects,certain insects do that like spiders and stuff
2,insects,too cold for his normal prey flying insects
3,insects,spiders and insects are members of the ants
4,insects,they eat mice insects and little birds sometimes
...,...,...
2108100,goners,if an avalanche comes we re goners here
2108101,goners,do not find a ledge we re goners
2108102,goners,an avalanche comes we re goners here too
2108103,perceives,at least that s the way he perceives


In [42]:
df_word_in_eight_list = df_word_in_eight.groupby(["word"])["eightgram"].apply(list).reset_index(name="eightgram")
df_word_in_eight_list

,word,eightgram
0,a,"[can i talk to you for a second, do not think ..."
1,aa,"[let me get a couple aa batteries bro, what ar..."
2,aaa,"[let s just say i am aaa affiliated, that thin..."
3,aaargh,[no aaargh at the back of the throat]
4,aah,"[my life be like ooh aah ooh ooh, aah take the..."
...,...,...
38771,zuko,"[she laid her eyes on you huh zuko, you guys g..."
38772,zulu,"[zulu that s the way you say zulu, zulu that s..."
38773,zurg,"[buzz you could have defeated zurg all along, ..."
38774,zurich,"[i graduate i have decided to leave zurich, i ..."


In [43]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "ninegram"
df_target = df_ninegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [44]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_nine = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_word_in_nine

,word,ninegram
0,cooking,as soon as i got home i started cooking
1,cooking,great cooking is not for the faint of heart
2,cooking,a little turtle cooking in his little turtle suit
3,cooking,like a little turtle cooking in his little turtle
4,cooking,i bet if i stopped cooking you d call
...,...,...
1314160,broadly,within the narrative that is broadly speaking ...
1314161,kerim,here s a snapshot kerim managed to get of
1314162,kerim,s a snapshot kerim managed to get of her
1314163,bubbe,bubbe i probably know a lot you do not


In [45]:
df_word_in_nine_list = df_word_in_nine.groupby(["word"])["ninegram"].apply(list).reset_index(name="ninegram")
df_word_in_nine_list

,word,ninegram
0,a,"[i do not think that s a good idea, our whole ..."
1,aa,"[there s aa man tied up in my room, a minute d..."
2,aaa,"[does that thing have a aaa a belt battery, th..."
3,aah,"[ooh aah take the world and paint it red, you ..."
4,aand,"[aand fix the toilet and clean up the kitchen,..."
...,...,...
34993,zuckerberg,[what was mr zuckerberg s ownership share dilu...
34994,zuko,"[so she laid her eyes on you huh zuko, you guy..."
34995,zurg,[an agent of zurg if i ever saw one]
34996,zurich,[when i graduate i have decided to leave zuric...


In [46]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "tengram"
df_target = df_tengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [47]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_ten = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_word_in_ten

,word,tengram
0,tracker,now we might pick it up on the tracker again
1,tracker,he s a good tracker and had a high iq
2,insects,it s too cold for his normal prey flying insects
3,insects,there s no food and few insects to feed on
4,tracker,i have not seen him and my tracker is blank
...,...,...
635169,scooping,scooping up artifacts she thinks might have pa...
635170,scooping,gangsters were just scooping money off the str...
635171,qtip,you stick a giant qtip down their throat and wait
635172,derives,supreme executive power derives from a mandate...


In [48]:
df_word_in_ten_list = df_word_in_ten.groupby(["word"])["tengram"].apply(list).reset_index(name="tengram")
df_word_in_ten_list

,word,tengram
0,a,"[i do not think that s such a good idea, when ..."
1,aa,"[wait a minute do not i know you from aa, ther..."
2,aaa,[does that thing have a aaa a belt battery pack]
3,aang,[how do you think avatar aang would have handl...
4,aaron,[i believe you testified you backed away from ...
...,...,...
28306,zorg,[if zorg really wants them he will have to neg...
28307,zorin,[about the same time zorin came over to the we...
28308,zucchini,[you and banner better not be playing hide the...
28309,zuckerberg,[what was mr zuckerberg s ownership share dilu...


In [49]:
dfs = [df_word_select, df_word_in_two_list, df_word_in_three_list, df_word_in_four_list, df_word_in_five_list, df_word_in_six_list, df_word_in_seven_list, df_word_in_eight_list, df_word_in_nine_list, df_word_in_ten_list]

In [50]:
df_word_merge = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='left'), dfs)
df_word_merge

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,you,102069964,"[you re, are you, do you, you know, thank you,...","[what are you, what do you, you do not, are yo...","[what are you doing, what do you mean, what do...","[what are you doing here, what are you talking...","[what the hell are you doing, i do not know wh...","[do not know what you re talking, i do not kno...","[do not know what you re talking about, i do n...","[i do not know what you re talking about, i ha...","[that s what i have been trying to tell you, y..."
1,i,94447074,"[i am, i do, i will, i have, i can, i know, i ...","[i do not, i am not, i am sorry, i can not, i ...","[i do not know, i do not want, i do not think,...","[i do not want to, i do not know what, i do no...","[i do not know what to, i do not know what you...","[i do not know what you re, i do not know what...","[i do not know what you re talking, i do not w...","[i do not know what you re talking about, i ha...","[that s what i have been trying to tell you, a..."
2,the,77481215,"[in the, s the, to the, on the, of the, what t...","[what s the, what the hell, that s the, it s t...","[what s the matter, the hell are you, what the...","[what the hell are you, the hell are you doing...","[what the hell are you doing, what s the matte...","[what the hell are you doing here, what the he...","[what the hell do you think you re, let s get ...","[what the hell do you think you re doing, hail...",[hail mary full of grace the lord is with thee...
3,to,58281119,"[want to, have to, to be, to the, to me, going...","[you want to, i want to, i have to, not want t...","[do not want to, i am going to, do not have to...","[i do not want to, you do not have to, do you ...","[i do not know what to, what do you want me to...","[what do you want me to do, i do not know what...","[i do not want to talk about it, can i talk to...","[how many times do i have to tell you, what do...","[that s what i have been trying to tell you, y..."
4,is,50852895,"[this is, is it, is not, what is, is that, it ...","[what is it, this is not, this is a, is not it...","[what the hell is, is that what you, this is n...","[the hell is going on, what the hell is going,...","[what the hell is going on, what the fuck is g...","[what the hell is wrong with you, what the hel...","[what the hell is that supposed to mean, is th...","[hail mary full of grace the lord is with, mar...",[hail mary full of grace the lord is with thee...
...,...,...,...,...,...,...,...,...,...,...,...
44995,pythons,630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44996,bubbe,630,NaN,[bubbe i probably],"[bubbe i probably know, a good job bubbe, me a...","[bubbe i probably know a, did a good job bubbe...","[bubbe i probably know a lot, you did a good j...",[bubbe i probably know a lot you],[bubbe i probably know a lot you do],[bubbe i probably know a lot you do not],NaN
44997,clucking,630,NaN,NaN,"[clucking like a hen, and even without cluckin...","[and even without clucking like, even without ...","[and even without clucking like a, even withou...","[and even without clucking like a hen, the chi...",[wish you two hens would quit your clucking],[i wish you two hens would quit your clucking],NaN
44998,komal,630,NaN,NaN,NaN,NaN,NaN,[you know what i am saying komal],NaN,NaN,NaN


In [51]:
#df_word_merge.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx", index=False)
df_word_merge.to_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.csv", index=False)

In [52]:
#df_word_text = pd.read_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx")
df_word_text = pd.read_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.csv")

In [53]:
df_word_merge2 = df_word_text[~(df_word_text["twogram"].isnull() & df_word_text["threegram"].isnull() & df_word_text["fourgram"].isnull() & df_word_text["fivegram"].isnull() & \
df_word_text["sixgram"].isnull() & df_word_text["sevengram"].isnull() & df_word_text["eightgram"].isnull() & df_word_text["ninegram"].isnull() & df_word_text["tengram"].isnull())]
df_word_merge2

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,you,102069964,"['you re', 'are you', 'do you', 'you know', 't...","['what are you', 'what do you', 'you do not', ...","['what are you doing', 'what do you mean', 'wh...","['what are you doing here', 'what are you talk...","['what the hell are you doing', 'i do not know...","['do not know what you re talking', 'i do not ...","['do not know what you re talking about', 'i d...","['i do not know what you re talking about', 'i...","['that s what i have been trying to tell you',..."
1,i,94447074,"['i am', 'i do', 'i will', 'i have', 'i can', ...","['i do not', 'i am not', 'i am sorry', 'i can ...","['i do not know', 'i do not want', 'i do not t...","['i do not want to', 'i do not know what', 'i ...","['i do not know what to', 'i do not know what ...","['i do not know what you re', 'i do not know w...","['i do not know what you re talking', 'i do no...","['i do not know what you re talking about', 'i...","['that s what i have been trying to tell you',..."
2,the,77481215,"['in the', 's the', 'to the', 'on the', 'of th...","['what s the', 'what the hell', 'that s the', ...","['what s the matter', 'the hell are you', 'wha...","['what the hell are you', 'the hell are you do...","['what the hell are you doing', 'what s the ma...","['what the hell are you doing here', 'what the...","['what the hell do you think you re', 'let s g...","['what the hell do you think you re doing', 'h...",['hail mary full of grace the lord is with the...
3,to,58281119,"['want to', 'have to', 'to be', 'to the', 'to ...","['you want to', 'i want to', 'i have to', 'not...","['do not want to', 'i am going to', 'do not ha...","['i do not want to', 'you do not have to', 'do...","['i do not know what to', 'what do you want me...","['what do you want me to do', 'i do not know w...","['i do not want to talk about it', 'can i talk...","['how many times do i have to tell you', 'what...","['that s what i have been trying to tell you',..."
4,is,50852895,"['this is', 'is it', 'is not', 'what is', 'is ...","['what is it', 'this is not', 'this is a', 'is...","['what the hell is', 'is that what you', 'this...","['the hell is going on', 'what the hell is goi...","['what the hell is going on', 'what the fuck i...","['what the hell is wrong with you', 'what the ...","['what the hell is that supposed to mean', 'is...","['hail mary full of grace the lord is with', '...",['hail mary full of grace the lord is with the...
...,...,...,...,...,...,...,...,...,...,...,...
44993,eragon,630,NaN,NaN,NaN,"['those walls hold death eragon', 'eragon stay...","['my story was about you eragon', 'future is f...","['the future is for the living eragon', 'erago...",['eragon this is more than about just us'],NaN,NaN
44996,bubbe,630,NaN,['bubbe i probably'],"['bubbe i probably know', 'a good job bubbe', ...","['bubbe i probably know a', 'did a good job bu...","['bubbe i probably know a lot', 'you did a goo...",['bubbe i probably know a lot you'],['bubbe i probably know a lot you do'],['bubbe i probably know a lot you do not'],NaN
44997,clucking,630,NaN,NaN,"['clucking like a hen', 'and even without cluc...","['and even without clucking like', 'even witho...","['and even without clucking like a', 'even wit...","['and even without clucking like a hen', 'the ...",['wish you two hens would quit your clucking'],['i wish you two hens would quit your clucking'],NaN
44998,komal,630,NaN,NaN,NaN,NaN,NaN,['you know what i am saying komal'],NaN,NaN,NaN


In [54]:
#df_word_merge2.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)
df_word_merge2.to_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.csv", index=False)

##### Twogram

In [20]:
# 2 Way Fast
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [21]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_two_in_three = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_three

,twogram,threegram
0,you seek,do you seek
1,you seek,what you seek
2,you seek,you seek is
3,you seek,you seek the
4,you seek,you seek to
...,...,...
1522135,creepy is,creepy is what
1522136,load for,a load for
1522137,load for,load for free
1522138,and whipped,and whipped cream


In [22]:
df_two_in_three_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_three_list

,twogram,threegram
0,a a,"[s a a, was a a, have a a, like a a, is a a, h..."
1,a and,"[a and b, a and c, a and a, teams a and, the a..."
2,a b,"[a b c, got a b, with a b, a b and, of a b, s ..."
3,a ba,[a ba in]
4,a babe,"[s a babe, what a babe, a babe in, like a babe..."
...,...,...
162346,zoned out,[zoned out for]
162347,zoom in,"[zoom in on, you zoom in, and zoom in, now zoo..."
162348,zoom zoom,[zoom zoom zoom]
162349,zulu xray,[yankee zulu xray]


In [23]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [24]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_four

,twogram,fourgram
0,one small,one small step for
1,one small,s just one small
2,one small,just one small problem
3,one small,there is one small
4,one small,is just one small
...,...,...
4017882,creepy is,of creepy is not
4017883,load for,take a load for
4017884,load for,a load for free
4017885,is oskar,my name is oskar


In [25]:
df_two_in_four_list = df_two_in_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_in_four_list

,twogram,fourgram
0,a a,"[it was a a, it s a a, that s a a, i have a a,..."
1,a and,"[teams a and b, a and b gun, a and b sentries,..."
2,a b,"[a b c d, i got a b, son of a b, a b b minus, ..."
3,a ba,[with a ba in]
4,a babe,"[she s a babe, you re a babe, a babe in arms, ..."
...,...,...
164634,zone is,"[white zone is for, the white zone is, zone is..."
164635,zoned out,"[zoned out for a, zoned out during that, must ..."
164636,zoom in,"[can you zoom in, zoom in on that, you zoom in..."
164637,zooming in,[i am zooming in]


In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_five = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_five

In [ ]:
df_two_in_five_list = df_two_in_five.groupby(["twogram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_two_in_five_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_six = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_six

In [ ]:
df_two_in_six_list = df_two_in_six.groupby(["twogram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_two_in_six_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_seven = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_seven

In [ ]:
df_two_in_seven_list = df_two_in_seven.groupby(["twogram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_two_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_eight = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_eight

In [ ]:
df_two_in_eight_list = df_two_in_eight.groupby(["twogram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_two_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_nine = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_nine

In [ ]:
df_two_in_nine_list = df_two_in_nine.groupby(["twogram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_two_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "tengram"
df_target = df_tengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_ten = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_ten

In [ ]:
df_two_in_ten_list = df_two_in_ten.groupby(["twogram"])["tengram"].apply(list).reset_index(name="tengram")
df_two_in_ten_list

In [ ]:
dfs2 = [df_twogram_select, df_two_in_three_list, df_two_in_four_list, df_two_in_five_list, df_two_in_six_list, df_two_in_seven_list, df_two_in_eight_list, df_two_in_nine_list, df_two_in_ten_list]

In [ ]:
df_twogram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='left'), dfs2)
df_twogram_merge

In [ ]:
#df_twogram_merge.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.xlsx", index=False)
df_twogram_merge.to_csv(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_twogram_text = pd.read_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.xlsx")
df_twogram_text = pd.read_csv(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.csv")

In [ ]:
df_twogram_merge2 = df_twogram_text[~(df_twogram_text["threegram"].isnull() & df_twogram_text["fourgram"].isnull() & df_twogram_text["fivegram"].isnull() & df_twogram_text["sixgram"].isnull() & \
df_twogram_text["sevengram"].isnull() & df_twogram_text["eightgram"].isnull() & df_twogram_text["ninegram"].isnull() & df_twogram_text["tengram"].isnull())]
df_twogram_merge2

In [ ]:
#df_twogram_merge2.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx", index=False)
df_twogram_merge2.to_csv(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.csv", index=False)

##### Threegram

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_three_in_four = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_four

In [ ]:
df_three_in_four_list = df_three_in_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_in_four_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_five = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_five

In [ ]:
df_three_in_five_list = df_three_in_five.groupby(["threegram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_three_in_five_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_six = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_six

In [ ]:
df_three_in_six_list = df_three_in_six.groupby(["threegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_three_in_six_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_seven = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_seven

In [ ]:
df_three_in_seven_list = df_three_in_seven.groupby(["threegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_three_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_eight = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_eight

In [ ]:
df_three_in_eight_list = df_three_in_eight.groupby(["threegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_three_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_nine = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_nine

In [ ]:
df_three_in_nine_list = df_three_in_nine.groupby(["threegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_three_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "tengram"
df_target = df_tengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_ten = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_ten

In [ ]:
df_three_in_ten_list = df_three_in_ten.groupby(["threegram"])["tengram"].apply(list).reset_index(name="tengram")
df_three_in_ten_list

In [ ]:
dfs3 = [df_threegram_select, df_three_in_four_list, df_three_in_five_list, df_three_in_six_list, df_three_in_seven_list, df_three_in_eight_list, df_three_in_nine_list, df_three_in_ten_list]

In [ ]:
df_threegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['threegram'], how='left'), dfs3)
df_threegram_merge

In [ ]:
#df_threegram_merge.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.xlsx", index=False)
df_threegram_merge.to_csv(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_threegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.xlsx")
df_threegram_text = pd.read_csv(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.csv")

In [ ]:
df_threegram_merge2 = df_threegram_text[~(df_threegram_text["fourgram"].isnull() & df_threegram_text["fivegram"].isnull() & df_threegram_text["sixgram"].isnull() & \
df_threegram_text["sevengram"].isnull() & df_threegram_text["eightgram"].isnull() & df_threegram_text["ninegram"].isnull() & df_threegram_text["tengram"].isnull())]
df_threegram_merge2

In [ ]:
#df_threegram_merge2.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx", index=False)
df_threegram_merge2.to_csv(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.csv", index=False)

##### Fourgram

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_four_in_five = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_five = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_four_in_five

In [ ]:
df_four_in_five_list = df_four_in_five.groupby(["fourgram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_four_in_five_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_six = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_four_in_six

In [ ]:
df_four_in_six_list = df_four_in_six.groupby(["fourgram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_four_in_six_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_seven = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_four_in_seven

In [ ]:
df_four_in_seven_list = df_four_in_seven.groupby(["fourgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_four_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_eight = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_four_in_eight

In [ ]:
df_four_in_eight_list = df_four_in_eight.groupby(["fourgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_four_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_nine = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_four_in_nine

In [ ]:
df_four_in_nine_list = df_four_in_nine.groupby(["fourgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_four_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "tengram"
df_target = df_tengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_ten = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_four_in_ten

In [ ]:
df_four_in_ten_list = df_four_in_ten.groupby(["fourgram"])["tengram"].apply(list).reset_index(name="tengram")
df_four_in_ten_list

In [ ]:
dfs4 = [df_fourgram_select, df_four_in_five_list, df_four_in_six_list, df_four_in_seven_list, df_four_in_eight_list, df_four_in_nine_list, df_four_in_ten_list]

In [ ]:
df_fourgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fourgram'], how='left'), dfs4)
df_fourgram_merge

In [ ]:
#df_fourgram_merge.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.xlsx", index=False)
df_fourgram_merge.to_csv(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_fourgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.xlsx")
df_fourgram_text = pd.read_csv(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.csv")

In [ ]:
df_fourgram_merge2 = df_fourgram_text[~(df_fourgram_text["fivegram"].isnull() & df_fourgram_text["sixgram"].isnull() & df_fourgram_text["sevengram"].isnull() & \
df_fourgram_text["eightgram"].isnull() & df_fourgram_text["ninegram"].isnull() & df_fourgram_text["tengram"].isnull())]
df_fourgram_merge2

In [ ]:
#df_fourgram_merge2.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx", index=False)
df_fourgram_merge2.to_csv(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.csv", index=False)

##### Fivegram

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_five_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_six = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_five_in_six

In [ ]:
df_five_in_six_list = df_five_in_six.groupby(["fivegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_five_in_six_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_seven = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_five_in_seven

In [ ]:
df_five_in_seven_list = df_five_in_seven.groupby(["fivegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_five_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_eight = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_five_in_eight

In [ ]:
df_five_in_eight_list = df_five_in_eight.groupby(["fivegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_five_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_nine = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_five_in_nine

In [ ]:
df_five_in_nine_list = df_five_in_nine.groupby(["fivegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_five_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "tengram"
df_target = df_tengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_ten = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_five_in_ten

In [ ]:
df_five_in_ten_list = df_five_in_ten.groupby(["fivegram"])["tengram"].apply(list).reset_index(name="tengram")
df_five_in_ten_list

In [ ]:
dfs5 = [df_fivegram_select, df_five_in_six_list, df_five_in_seven_list, df_five_in_eight_list, df_five_in_nine_list, df_five_in_ten_list]

In [ ]:
df_fivegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fivegram'], how='left'), dfs5)
df_fivegram_merge

In [ ]:
#df_fivegram_merge.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.xlsx", index=False)
df_fivegram_merge.to_csv(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_fivegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.xlsx")
df_fivegram_text = pd.read_csv(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.csv")

In [ ]:
df_fivegram_merge2 = df_fivegram_text[~(df_fivegram_text["sixgram"].isnull() & df_fivegram_text["sevengram"].isnull() & \
df_fivegram_text["eightgram"].isnull() & df_fivegram_text["ninegram"].isnull() & df_fivegram_text["tengram"].isnull())]
df_fivegram_merge2

In [ ]:
#df_fivegram_merge2.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx", index=False)
df_fivegram_merge2.to_csv(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.csv", index=False)

##### Sixgram

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_six_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_six_in_seven = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_six_in_seven

In [ ]:
df_six_in_seven_list = df_six_in_seven.groupby(["sixgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_six_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_eight = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_six_in_eight

In [ ]:
df_six_in_eight_list = df_six_in_eight.groupby(["sixgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_six_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_nine = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_six_in_nine

In [ ]:
df_six_in_nine_list = df_six_in_nine.groupby(["sixgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_six_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "tengram"
df_target = df_tengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_ten = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_six_in_ten

In [ ]:
df_six_in_ten_list = df_six_in_ten.groupby(["sixgram"])["tengram"].apply(list).reset_index(name="tengram")
df_six_in_ten_list

In [ ]:
dfs6 = [df_sixgram_select, df_six_in_seven_list, df_six_in_eight_list, df_six_in_nine_list, df_six_in_ten_list]

In [ ]:
df_sixgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sixgram'], how='left'), dfs6)
df_sixgram_merge

In [ ]:
#df_sixgram_merge.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.xlsx", index=False)
df_sixgram_merge.to_csv(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_sixgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.xlsx")
df_sixgram_text = pd.read_csv(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.csv")

In [ ]:
df_sixgram_merge2 = df_sixgram_text[~(df_sixgram_text["sevengram"].isnull() & df_sixgram_text["eightgram"].isnull() & df_sixgram_text["ninegram"].isnull() & df_sixgram_text["tengram"].isnull())]
df_sixgram_merge2

In [ ]:
#df_sixgram_merge2.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx", index=False)
df_sixgram_merge2.to_csv(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.csv", index=False)

##### Sevengram

In [99]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [100]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_seven_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_seven_in_eight = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_seven_in_eight

,sevengram,eightgram
0,size onların günlük hayatı hakkında biraz bilgi,size onların günlük hayatı hakkında biraz bilg...
1,size onların günlük hayatı hakkında biraz bilgi,tablo size onların günlük hayatı hakkında bira...
2,dobby bundan ancak efendisi ona giyecek verirse,dobby bundan ancak efendisi ona giyecek verirs...
3,yönetici oldum bir sürü siyasi yardım alıyorum,artık yönetici oldum bir sürü siyasi yardım al...
4,bu kadar güzel bir haber almadan geçiriyorlar,hayatlarını bu kadar güzel bir haber almadan g...
...,...,...
125750,al o zaman silahını al tabancanı ve,al o zaman silahını al tabancanı ve çek
125751,dört gözlü bir zenci hakkındaki kitabı okumak,neden dört gözlü bir zenci hakkındaki kitabı o...
125752,dört gözlü bir zenci hakkındaki kitabı okumak,dört gözlü bir zenci hakkındaki kitabı okumak ...
125753,devlet başkanı belki yarın gelir ama bunlar,devlet başkanı belki yarın gelir ama bunlar ço...


In [101]:
df_seven_in_eight_list = df_seven_in_eight.groupby(["sevengram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_seven_in_eight_list

,sevengram,eightgram
0,a annem dedi ki seni teşvik etmek,"[hepsi a annem dedi ki seni teşvik etmek, a an..."
1,a b c d diye alfabetik gider,[alanları a b c d diye alfabetik gider]
2,a e d a l u s,[d a e d a l u s]
3,a evet şu güney sokaktaki arabaları çektikleri,[a evet şu güney sokaktaki arabaları çektikler...
4,a n d o r u m,[p a n d o r u m]
...,...,...
101897,şüphesiz ki yapamazsan daha başlamadan yenilgi...,[şüphesiz ki yapamazsan daha başlamadan yenilg...
101898,şüphesiz o da herkes gibi bir avukat,[şüphesiz o da herkes gibi bir avukat alt]
101899,şüphesiz şimdiye kadar gördüğüm en kötü korsan,[bu şüphesiz şimdiye kadar gördüğüm en kötü ko...
101900,şık bir şeyler gir ve elini çabuk,"[şık bir şeyler gir ve elini çabuk tut, üstüne..."


In [102]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [103]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_nine = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_seven_in_nine

,sevengram,ninegram
0,size onların günlük hayatı hakkında biraz bilgi,bu tablo size onların günlük hayatı hakkında b...
1,size onların günlük hayatı hakkında biraz bilgi,tablo size onların günlük hayatı hakkında bira...
2,bu kadar güzel bir haber almadan geçiriyorlar,tüm hayatlarını bu kadar güzel bir haber almad...
3,arkadaşın arkadaşa yapmayacağı şeyler listesin...,bu arkadaşın arkadaşa yapmayacağı şeyler liste...
4,arkadaşın arkadaşa yapmayacağı şeyler listesin...,çünkü bu arkadaşın arkadaşa yapmayacağı şeyler...
...,...,...
70703,özel olarak çünkü bu sadece bir dosta,özel olarak çünkü bu sadece bir dosta söylenec...
70704,akşam üçüncü ve muhtemelen son olarak yerde,atom bu akşam üçüncü ve muhtemelen son olarak ...
70705,herifi durdurmak için sadece bir saat kırk,bu herifi durdurmak için sadece bir saat kırk ...
70706,herifi durdurmak için sadece bir saat kırk,herifi durdurmak için sadece bir saat kırk dak...


In [104]:
df_seven_in_nine_list = df_seven_in_nine.groupby(["sevengram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_seven_in_nine_list

,sevengram,ninegram
0,a annem dedi ki seni teşvik etmek,[hepsi a annem dedi ki seni teşvik etmek lazım]
1,a b c d diye alfabetik gider,[park alanları a b c d diye alfabetik gider]
2,a onlardan biri olabilir ve b hala,[çünkü a onlardan biri olabilir ve b hala kızg...
3,a r i s y e n,[i t a r i s y e n]
4,a takımı değil belki ama elimizden gelenin,[a takımı değil belki ama elimizden gelenin en...
...,...,...
58700,şüpheliyi suç mahalline getir ve kendini ele,[şüpheliyi suç mahalline getir ve kendini ele ...
58701,şüphen varsa beni vur parayı al ve,[şüphen varsa beni vur parayı al ve güzelce uy...
58702,şüphesiz ki yapamazsan daha başlamadan yenilgi...,[şüphesiz ki yapamazsan daha başlamadan yenilg...
58703,şüphesiz o da herkes gibi bir avukat,[şüphesiz o da herkes gibi bir avukat alt edeb...


In [105]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "tengram"
df_target = df_tengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [106]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_ten = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_seven_in_ten

,sevengram,tengram
0,bu kadar güzel bir haber almadan geçiriyorlar,bazıları tüm hayatlarını bu kadar güzel bir ha...
1,size onların günlük hayatı hakkında biraz bilgi,bu tablo size onların günlük hayatı hakkında b...
2,bir iki üç dört beş altı yedi,bir iki üç dört beş altı yedi sekiz dokuz on
3,bir iki üç dört beş altı yedi,bir iki üç dört beş altı yedi sekiz dokuz dön
4,bir iki üç dört beş altı yedi,orada bir iki üç dört beş altı yedi tane var
...,...,...
24352,seni aşağıya indireceğiz oyunu yakalayacaksın ...,yavaşça seni aşağıya indireceğiz oyunu yakalay...
24353,şeytana olan aşkını nasıl da dile getiriyor,onu bir dinle şeytana olan aşkını nasıl da dil...
24354,özel olarak çünkü bu sadece bir dosta,özel olarak çünkü bu sadece bir dosta söylenec...
24355,herifi durdurmak için sadece bir saat kırk,bu herifi durdurmak için sadece bir saat kırk ...


In [107]:
df_seven_in_ten_list = df_seven_in_ten.groupby(["sevengram"])["tengram"].apply(list).reset_index(name="tengram")
df_seven_in_ten_list

,sevengram,tengram
0,a onlardan biri olabilir ve b hala,[çünkü a onlardan biri olabilir ve b hala kızg...
1,a r i s y e n,[g i t a r i s y e n]
2,a takımı değil belki ama elimizden gelenin,[a takımı değil belki ama elimizden gelenin en...
3,abc gündüz ve gece için yeni evli,[abc gündüz ve gece için yeni evli oyunu nu se...
4,abd hava üssünde patlayacak atom bombası daha,[abd hava üssünde patlayacak atom bombası daha...
...,...,...
24198,şöyle söyleyeyim sanki şu an burada dikiliyor,[şöyle söyleyeyim sanki şu an burada dikiliyor...
24199,şöyle yeşil nakite ihtiyacı vardır anlıyorsun ya,[bilirsin bir kardeş şöyle yeşil nakite ihtiya...
24200,şükürler olsun ki o zamandan bu yana,[şükürler olsun ki o zamandan bu yana uzun yol...
24201,şüpheli birinin beyin ameliyatı yapmasına izin...,[elbette siyasi olarak şüpheli birinin beyin a...


In [108]:
dfs7 = [df_sevengram_select, df_seven_in_eight_list, df_seven_in_nine_list, df_seven_in_ten_list]

In [109]:
df_sevengram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sevengram'], how='left'), dfs7)
df_sevengram_merge

,sevengram,frequency,eightgram,ninegram,tengram
0,bir iki üç dört beş altı yedi,1455,"[bir iki üç dört beş altı yedi sekiz, ve bir i...","[bir iki üç dört beş altı yedi sekiz dokuz, ve...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
1,iki üç dört beş altı yedi sekiz,1080,"[bir iki üç dört beş altı yedi sekiz, iki üç d...","[bir iki üç dört beş altı yedi sekiz dokuz, ik...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
2,bana söylemek istediğin bir şey var mı,752,"[önce bana söylemek istediğin bir şey var mı, ...",[çıkmadan önce bana söylemek istediğin bir şey...,[arabadan çıkmadan önce bana söylemek istediği...
3,yedi altı beş dört üç iki bir,596,"[sekiz yedi altı beş dört üç iki bir, yedi alt...","[dokuz sekiz yedi altı beş dört üç iki bir, se...","[on dokuz sekiz yedi altı beş dört üç iki bir,..."
4,sekiz yedi altı beş dört üç iki,551,"[dokuz sekiz yedi altı beş dört üç iki, sekiz ...","[dokuz sekiz yedi altı beş dört üç iki bir, on...","[on dokuz sekiz yedi altı beş dört üç iki bir,..."
...,...,...,...,...,...
141654,akşam üçüncü ve muhtemelen son olarak yerde,9,[bu akşam üçüncü ve muhtemelen son olarak yerde],[atom bu akşam üçüncü ve muhtemelen son olarak...,NaN
141655,herifi durdurmak için sadece bir saat kırk,9,[herifi durdurmak için sadece bir saat kırk da...,[bu herifi durdurmak için sadece bir saat kırk...,[bu herifi durdurmak için sadece bir saat kırk...
141656,görücü usulü evlenmek daha çok işe yarıyor,9,[bazen görücü usulü evlenmek daha çok işe yarı...,NaN,NaN
141657,konuda çok yanlış olduğundan kesinlikle eminim...,9,[bu konuda çok yanlış olduğundan kesinlikle em...,NaN,NaN


In [110]:
#df_sevengram_merge.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.xlsx", index=False)
df_sevengram_merge.to_csv(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.csv", index=False)

In [111]:
#df_sevengram_text = pd.read_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.xlsx")
df_sevengram_text = pd.read_csv(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.csv")

In [112]:
df_sevengram_merge2 = df_sevengram_text[~(df_sevengram_text["eightgram"].isnull() & df_sevengram_text["ninegram"].isnull() & df_sevengram_text["tengram"].isnull())]
df_sevengram_merge2

,sevengram,frequency,eightgram,ninegram,tengram
0,bir iki üç dört beş altı yedi,1455,"['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,iki üç dört beş altı yedi sekiz,1080,"['bir iki üç dört beş altı yedi sekiz', 'iki ü...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,bana söylemek istediğin bir şey var mı,752,['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,yedi altı beş dört üç iki bir,596,"['sekiz yedi altı beş dört üç iki bir', 'yedi ...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
4,sekiz yedi altı beş dört üç iki,551,"['dokuz sekiz yedi altı beş dört üç iki', 'sek...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
...,...,...,...,...,...
141654,akşam üçüncü ve muhtemelen son olarak yerde,9,['bu akşam üçüncü ve muhtemelen son olarak yer...,['atom bu akşam üçüncü ve muhtemelen son olara...,NaN
141655,herifi durdurmak için sadece bir saat kırk,9,['herifi durdurmak için sadece bir saat kırk d...,['bu herifi durdurmak için sadece bir saat kır...,['bu herifi durdurmak için sadece bir saat kır...
141656,görücü usulü evlenmek daha çok işe yarıyor,9,['bazen görücü usulü evlenmek daha çok işe yar...,NaN,NaN
141657,konuda çok yanlış olduğundan kesinlikle eminim...,9,['bu konuda çok yanlış olduğundan kesinlikle e...,NaN,NaN


In [113]:
#df_sevengram_merge2.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx", index=False)
df_sevengram_merge2.to_csv(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.csv", index=False)

##### Eightgram

In [87]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [88]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_eight_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_eight_in_nine = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_eight_in_nine

,eightgram,ninegram
0,tür bir oyun olduğunu öğrenmek istiyorsun nası...,ne tür bir oyun olduğunu öğrenmek istiyorsun n...
1,tür bir oyun olduğunu öğrenmek istiyorsun nası...,tür bir oyun olduğunu öğrenmek istiyorsun nası...
2,hey sizin istediğiniz bildik yollar öyle yapal...,hey sizin istediğiniz bildik yollar öyle yapal...
3,adımı vermeyi teklif etmeniz benim için büyük bir,adımı vermeyi teklif etmeniz benim için büyük ...
4,adam işe çıktı operasyonu berbat etti bir daha,adam işe çıktı operasyonu berbat etti bir daha...
...,...,...
80530,hong konga gidip bütün çeteyi temizleyecek ces...,hong konga gidip bütün çeteyi temizleyecek ces...
80531,ülkenin beni ne duruma getirdiğini bir tek sen,bu ülkenin beni ne duruma getirdiğini bir tek sen
80532,ülkenin beni ne duruma getirdiğini bir tek sen,ülkenin beni ne duruma getirdiğini bir tek sen...
80533,bayan şu olaylar çerçevesinde kocanızın sandığ...,bayan şu olaylar çerçevesinde kocanızın sandığ...


In [89]:
df_eight_in_nine_list = df_eight_in_nine.groupby(["eightgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_eight_in_nine_list

,eightgram,ninegram
0,a annem dedi ki seni teşvik etmek lazım,[hepsi a annem dedi ki seni teşvik etmek lazım]
1,a kenarı b kenarı ve c kenarına eşittir,[üçgeni a kenarı b kenarı ve c kenarına eşittir]
2,a noktasından b noktasına olabildiğince hızlı ...,[a noktasından b noktasına olabildiğince hızlı...
3,a onlardan biri olabilir ve b hala kızgın,[çünkü a onlardan biri olabilir ve b hala kızg...
4,a takımı değil belki ama elimizden gelenin en,[a takımı değil belki ama elimizden gelenin en...
...,...,...
70432,şüphesiz ki yapamazsan daha başlamadan yenilgi...,[şüphesiz ki yapamazsan daha başlamadan yenilg...
70433,şüphesiz o da herkes gibi bir avukat alt,[şüphesiz o da herkes gibi bir avukat alt edeb...
70434,şüphesiz tek başına bu bile çok büyük bir,[şüphesiz tek başına bu bile çok büyük bir suç...
70435,şüpheye yer vermeyecek şekilde gözlem altına a...,[beni şüpheye yer vermeyecek şekilde gözlem al...


In [90]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "tengram"
df_target = df_tengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [91]:
#search_list = list(searchlist)
result_list = list(resultlist)
df_eight_in_ten = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_eight_in_ten

,eightgram,tengram
0,bir farenin duvardan geçtiği gibi kapının altı...,bir farenin duvardan geçtiği gibi kapının altı...
1,bir sürü şey için söz veriyorsun ama hiçbirini,bana bir sürü şey için söz veriyorsun ama hiçb...
2,herkesten daha iyi biliyorsun ki memeler kimse...,buradaki herkesten daha iyi biliyorsun ki meme...
3,ben bunu bana yapacağını ya da yapabileceğini hiç,ama ben bunu bana yapacağını ya da yapabileceğ...
4,erkeğin evini ve sevdiklerini koruma hakkı var...,her erkeğin evini ve sevdiklerini koruma hakkı...
...,...,...
31027,bu bok en uzun süre ne kadar etki,bu bok en uzun süre ne kadar etki gösterdi sende
31028,şu anda telefona gelemez ama beni bıraktığınız...,joshua şu anda telefona gelemez ama beni bırak...
31029,istediğim elini içeri sokman ve kabloyu nazikç...,senden istediğim elini içeri sokman ve kabloyu...
31030,insan diğer bir insanın işine çomak sokmak zor...,neden bir insan diğer bir insanın işine çomak ...


In [92]:
df_eight_in_ten_list = df_eight_in_ten.groupby(["eightgram"])["tengram"].apply(list).reset_index(name="tengram")
df_eight_in_ten_list

,eightgram,tengram
0,a kenarı b kenarı ve c kenarına eşittir,[abc üçgeni a kenarı b kenarı ve c kenarına eş...
1,a onlardan biri olabilir ve b hala kızgın,[çünkü a onlardan biri olabilir ve b hala kızg...
2,a takımı değil belki ama elimizden gelenin en,[a takımı değil belki ama elimizden gelenin en...
3,abc üçgeni a kenarı b kenarı ve c,[abc üçgeni a kenarı b kenarı ve c kenarına eş...
4,abd hava üssünde patlayacak atom bombası daha çok,[abd hava üssünde patlayacak atom bombası daha...
...,...,...
30966,şükürler olsun ki bu kiliseyi koruması için bi...,[tanrıya şükürler olsun ki bu kiliseyi korumas...
30967,şükürler olsun ki o zamandan bu yana uzun,[şükürler olsun ki o zamandan bu yana uzun yol...
30968,şüphen varsa beni vur parayı al ve güzelce,[bundan şüphen varsa beni vur parayı al ve güz...
30969,şüphesiz erdemli bir lider sadece küçük bir ar...,[şüphesiz erdemli bir lider sadece küçük bir a...


In [93]:
dfs8 = [df_eightgram_select, df_eight_in_nine_list, df_eight_in_ten_list]

In [94]:
df_eightgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['eightgram'], how='left'), dfs8)
df_eightgram_merge

,eightgram,frequency,ninegram,tengram
0,bir iki üç dört beş altı yedi sekiz,1060,"[bir iki üç dört beş altı yedi sekiz dokuz, ve...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
1,dokuz sekiz yedi altı beş dört üç iki,513,"[dokuz sekiz yedi altı beş dört üç iki bir, on...","[on dokuz sekiz yedi altı beş dört üç iki bir,..."
2,sekiz yedi altı beş dört üç iki bir,497,"[dokuz sekiz yedi altı beş dört üç iki bir, se...","[on dokuz sekiz yedi altı beş dört üç iki bir,..."
3,iki üç dört beş altı yedi sekiz dokuz,411,"[bir iki üç dört beş altı yedi sekiz dokuz, ik...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
4,üç dört beş altı yedi sekiz dokuz on,329,"[iki üç dört beş altı yedi sekiz dokuz on, üç ...","[bir iki üç dört beş altı yedi sekiz dokuz on,..."
...,...,...,...,...
109469,ve bunu her yaptığında takım arkadaşlarından b...,7,NaN,NaN
109470,gizli gizli dolaşan genç bir haham imajı canla...,7,NaN,NaN
109471,için tek yol kahrolası paramı bana geri vermen,7,NaN,NaN
109472,teşkilatın görev kapsamında olsa bile görev se...,7,NaN,NaN


In [95]:
#df_eightgram_merge.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.xlsx", index=False)
df_eightgram_merge.to_csv(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.csv", index=False)

In [96]:
#df_eightgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.xlsx")
df_eightgram_text = pd.read_csv(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.csv")

In [97]:
df_eightgram_merge2 = df_eightgram_text[~(df_eightgram_text["ninegram"].isnull() & df_eightgram_text["tengram"].isnull())]
df_eightgram_merge2

,eightgram,frequency,ninegram,tengram
0,bir iki üç dört beş altı yedi sekiz,1060,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,dokuz sekiz yedi altı beş dört üç iki,513,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
2,sekiz yedi altı beş dört üç iki bir,497,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
3,iki üç dört beş altı yedi sekiz dokuz,411,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
4,üç dört beş altı yedi sekiz dokuz on,329,"['iki üç dört beş altı yedi sekiz dokuz on', '...",['bir iki üç dört beş altı yedi sekiz dokuz on...
...,...,...,...,...
109446,bu bok en uzun süre ne kadar etki,7,NaN,['bu bok en uzun süre ne kadar etki gösterdi s...
109448,şu anda telefona gelemez ama beni bıraktığınız...,7,NaN,['joshua şu anda telefona gelemez ama beni bır...
109455,istediğim elini içeri sokman ve kabloyu nazikç...,7,NaN,['senden istediğim elini içeri sokman ve kablo...
109462,insan diğer bir insanın işine çomak sokmak zor...,7,NaN,['neden bir insan diğer bir insanın işine çoma...


In [98]:
#df_eightgram_merge2.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx", index=False)
df_eightgram_merge2.to_csv(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.csv", index=False)

##### Ninegram

In [78]:
# parameter
source_wordgroup_list = ninegram_list
search_column_name = "ninegram"
target_column = "tengram"
df_target = df_tengram_select

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)].iloc[0:1000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [79]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_nine_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_nine_in_ten = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_nine_in_ten

,ninegram,tengram
0,ve büyük oğlu öldürülmeden önceki akşamüstü on...,leeds ve büyük oğlu öldürülmeden önceki akşamü...
1,kendinizi yola atıp koca şehrin bütün sorunlar...,kendinizi yola atıp koca şehrin bütün sorunlar...
2,hastayı kurtarmak için ne kadarını kesmek zoru...,böylece hastayı kurtarmak için ne kadarını kes...
3,uzun zamandır çalışıyorum ve kazandığım parayl...,çünkü uzun zamandır çalışıyorum ve kazandığım ...
4,muhtemelen bana gönderdikleri beş kutuyu da öd...,şimdi muhtemelen bana gönderdikleri beş kutuyu...
...,...,...
20255,başkalarının yaptığı ama benim yapamadığım şey...,başkalarının yaptığı ama benim yapamadığım şey...
20256,lordun şefe söyle onun ne yardımını ne de iyil...,lordun şefe söyle onun ne yardımını ne de iyil...
20257,kabalık ve kendini bilmek nadiren iyi bir ikil...,ama kabalık ve kendini bilmek nadiren iyi bir ...
20258,dergi haberi sağlayacağım dedim ve seni bir de...,sana dergi haberi sağlayacağım dedim ve seni b...


In [80]:
df_nine_in_ten_list = df_nine_in_ten.groupby(["ninegram"])["tengram"].apply(list).reset_index(name="tengram")
df_nine_in_ten_list

,ninegram,tengram
0,a onlardan biri olabilir ve b hala kızgın olab...,[çünkü a onlardan biri olabilir ve b hala kızg...
1,a takımı değil belki ama elimizden gelenin en ...,[a takımı değil belki ama elimizden gelenin en...
2,abc üçgeni a kenarı b kenarı ve c kenarına,[abc üçgeni a kenarı b kenarı ve c kenarına eş...
3,abd hava üssünde patlayacak atom bombası daha ...,[abd hava üssünde patlayacak atom bombası daha...
4,abd hükümeti ile alakalı gizli görevler için y...,[abd hükümeti ile alakalı gizli görevler için ...
...,...,...
20237,şükran gününde ve noelde masada sana da bir yer,[şükran gününde ve noelde masada sana da bir y...
20238,şükürler olsun ki bu kiliseyi koruması için bi...,[tanrıya şükürler olsun ki bu kiliseyi korumas...
20239,şükürler olsun ki o zamandan bu yana uzun yollar,[şükürler olsun ki o zamandan bu yana uzun yol...
20240,şüphen varsa beni vur parayı al ve güzelce uyu,[bundan şüphen varsa beni vur parayı al ve güz...


In [81]:
dfs9 = [df_ninegram_select, df_nine_in_ten_list]

In [82]:
df_ninegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['ninegram'], how='left'), dfs9)
df_ninegram_merge

,ninegram,frequency,tengram
0,dokuz sekiz yedi altı beş dört üç iki bir,463,"[on dokuz sekiz yedi altı beş dört üç iki bir,..."
1,bir iki üç dört beş altı yedi sekiz dokuz,405,"[bir iki üç dört beş altı yedi sekiz dokuz on,..."
2,iki üç dört beş altı yedi sekiz dokuz on,287,[bir iki üç dört beş altı yedi sekiz dokuz on]
3,on dokuz sekiz yedi altı beş dört üç iki,277,[on dokuz sekiz yedi altı beş dört üç iki bir]
4,ve sen herkesin seni tanıdığı o yerde olmak is...,134,NaN
...,...,...,...
40393,artık sana kan nakli yapacak doktor hoffman da...,7,NaN
40394,senin ülken için potansiyel terörist teröristi...,7,NaN
40395,sabah dokuz gibi geçerken uğrarım bu yüzden ön...,7,[sabah dokuz gibi geçerken uğrarım bu yüzden ö...
40396,herkes benim iyi adam olduğuma inanmakta bu ka...,7,[neden herkes benim iyi adam olduğuma inanmakt...


In [83]:
#df_ninegram_merge.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.xlsx", index=False)
df_ninegram_merge.to_csv(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.csv", index=False)

In [84]:
#df_ninegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.xlsx")
df_ninegram_text = pd.read_csv(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.csv")

In [85]:
df_ninegram_merge2 = df_ninegram_text[~(df_ninegram_text["tengram"].isnull())]
df_ninegram_merge2

,ninegram,frequency,tengram
0,dokuz sekiz yedi altı beş dört üç iki bir,463,['on dokuz sekiz yedi altı beş dört üç iki bir...
1,bir iki üç dört beş altı yedi sekiz dokuz,405,['bir iki üç dört beş altı yedi sekiz dokuz on...
2,iki üç dört beş altı yedi sekiz dokuz on,287,['bir iki üç dört beş altı yedi sekiz dokuz on']
3,on dokuz sekiz yedi altı beş dört üç iki,277,['on dokuz sekiz yedi altı beş dört üç iki bir']
6,i t a r i s y e n,128,['g i t a r i s y e n']
...,...,...,...
40382,sevgili kızım aşık bir kadın için bile mantıks...,7,['sevgili kızım aşık bir kadın için bile mantı...
40391,çene çalmayı bir yana bırakıp bu işi gerçekten...,7,['çene çalmayı bir yana bırakıp bu işi gerçekt...
40392,merak etme böyle bir şey yapmamı istemediğinde...,7,['merak etme böyle bir şey yapmamı istemediğin...
40395,sabah dokuz gibi geçerken uğrarım bu yüzden ön...,7,['sabah dokuz gibi geçerken uğrarım bu yüzden ...


In [86]:
#df_ninegram_merge2.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx", index=False)
df_ninegram_merge2.to_csv(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.csv", index=False)

#### Copy Move And Delete

In [55]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result*")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

['English_Word_Text_Selection_Result.csv']

In [56]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [57]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass